### start

In [ ]:
# pyright: reportMissingImports=false
# pyright: reportMissingModuleSource=false

import uuid
import random
import hashlib
import os
import sys
import gc
import time
import copy
import logging
import re
from itertools import chain,combinations
import pdb
import math
from pathlib import Path
import pickle
import cloudpickle
import datetime
from datetime import datetime,timedelta
import json
import warnings
import yaml
from typing import Dict,Union,List,Any,Tuple
import pytest
import importlib
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import IPython
import requests
import warnings
import boto3
import matplotlib as plt

# load_dotenv(Path("../../../Local/.env"))

# Custom format function for displaying |numbers/
pd.set_option('display.float_format', lambda x: f'{x:.12g}')
# pd.reset_option('display.float_format')

# Suppress warnings
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'
warnings.filterwarnings('ignore', message='.*frozen modules.*')
warnings.filterwarnings("ignore", message="MallocStackLogging")

# silence pygame donation request
os.environ['PYGAME_HIDE_SUPPORT_PROMPT'] = "hide"
os.environ['LOGGING_FILE'] = "../../../Local/logs/wallet_modeling.log"
os.environ['NOTIFICATION_SOUNDS_DIR'] = "../../../Local"

# Dark mode charts
plt.rcParams['figure.facecolor'] = '#181818'  # Custom background color (dark gray in this case)
plt.rcParams['axes.facecolor'] = '#181818'
plt.rcParams['text.color'] = '#afc6ba'
plt.rcParams['axes.labelcolor'] = '#afc6ba'
plt.rcParams['xtick.color'] = '#afc6ba'
plt.rcParams['ytick.color'] = '#afc6ba'
plt.rcParams['axes.titlecolor'] = '#afc6ba'
plt.rcParams['text.usetex'] = False
plt.rcParams['mathtext.default'] = 'regular'

import utils as u
# Set the custom error handler
ipython = IPython.get_ipython()
# ipython.set_custom_exc((Exception,), u.notify_on_failure)

player = u.AmbientPlayer()
player.stop_all_players()

# configure logger
logger = u.setup_notebook_logger('../logs/notebook_logs.log')
logger.setLevel(logging.INFO)

# import local modules
# pyright: reportMissingImports=false
sys.path.append('..//src')

import sagemaker_wallets.wallet_modeler as wm
import sagemaker_wallets.wallet_preprocessor as wp
import sagemaker_wallets.workflow_orchestrator as wo
from utils import ConfigError
import utils as u

# reload all modules
modules = [
    wm, wp, wo,
    u
]

from IPython.core import ultratb
ultratb.VerboseTB._tb_highlight = "bg:#b45827"


# load all configs
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


u.export_code(
    code_directories=[
        'sagemaker_wallets',
    ],
    # include_config = True,
    # ipynb_notebook = 'DDA-769 coin model score dist toggle.ipynb'
    output_file="temp/sagemaker_code.py"
)

[importlib.reload(module) for module in modules]
u.notify('retro')

logger.milestone("Good morning, let's get to work")

[15/Jul/25 21:02:48] INFO [credentials.load:1352] Found credentials in shared credentials file: ~/.aws/credentials


sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/jeremymeadow/Library/Application Support/sagemaker/config.yaml


[15/Jul/25 21:02:48] INFO [utils.export_code:1667] Consolidation complete. All files are saved in temp/sagemaker_code.py
[15/Jul/25 21:02:48] MILESTONE [3893980178.<module>:114] Good morning, let's get to work


# Initial ETLs

# Code begins

### Initiate orchestrator

In [2]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


workflow_orch = wo.WalletWorkflowOrchestrator(sage_wallets_config)

### Load, preprocess, and upload data

#### load data

In [ ]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))

date_suffixes = [
    '231107',
    '240306'
]
workflow_orch.load_training_data(date_suffixes)

##### inspect features

In [ ]:
# Create combined NaN count and describe statistics
nan_counts = workflow_orch.training_data['x_train'].isna().sum()
describe_stats = workflow_orch.training_data['x_train'].describe().T

# Combine into single DataFrame
combined_stats = pd.concat([
    nan_counts.rename('nan_count'),
    describe_stats
], axis=1).sort_index()

u.display_full(combined_stats.sort_index())

#### preprocess data

In [ ]:
preprocessor = wp.SageWalletsPreprocessor(sage_wallets_config)
preprocessed_data = preprocessor.preprocess_training_data(workflow_orch.training_data)

#### upload data

In [ ]:
# Upload and retrieve URIs
workflow_orch.upload_training_data(preprocessed_data, overwrite_existing=True)
s3_uris = workflow_orch.retrieve_training_data_uris(date_suffixes)

### Train Model

#### retrieve uris

In [13]:
date_suffixes = [
    '231107',
    '240306'
]

# Generate URIs for specific dates without any prior setup
s3_uris = workflow_orch.retrieve_training_data_uris(date_suffixes)
s3_uris

{'231107': {'train': 's3://wallet-training-data/training-data-preprocessed/dda-897-td-col-ordering/231107/train_cw_crypto_net_gain_crypto_inflows_winsorized.csv',
  'test': 's3://wallet-training-data/training-data-preprocessed/dda-897-td-col-ordering/231107/test_cw_crypto_net_gain_crypto_inflows_winsorized.csv',
  'eval': 's3://wallet-training-data/training-data-preprocessed/dda-897-td-col-ordering/231107/eval_cw_crypto_net_gain_crypto_inflows_winsorized.csv',
  'val': 's3://wallet-training-data/training-data-preprocessed/dda-897-td-col-ordering/231107/val_cw_crypto_net_gain_crypto_inflows_winsorized.csv'},
 '240306': {'train': 's3://wallet-training-data/training-data-preprocessed/dda-897-td-col-ordering/240306/train_cw_crypto_net_gain_crypto_inflows_winsorized.csv',
  'test': 's3://wallet-training-data/training-data-preprocessed/dda-897-td-col-ordering/240306/test_cw_crypto_net_gain_crypto_inflows_winsorized.csv',
  'eval': 's3://wallet-training-data/training-data-preprocessed/dda-897

#### train model

In [10]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


modeler = wm.WalletModeler(sage_wallets_config, s3_uris)
date_suffix = date_suffixes[0]



In [9]:
modeling_results = modeler.train_model(date_suffix)

[15/Jul/25 16:21:35] INFO [wallet_modeler.train_model:63] Starting SageMaker XGBoost training
[15/Jul/25 16:21:35] INFO [image_uris._processor:530] Ignoring unnecessary instance type: None.
[15/Jul/25 16:21:35] INFO [wallet_modeler.train_model:150] Launching training job: wallet-xgb-dda-897-td-col-ordering-231107-20250715-162135
[15/Jul/25 16:21:35] INFO [wallet_modeler.train_model:151] Model output path: s3://wallet-training-data/sagemaker-models/dda-897-td-col-ordering/
[15/Jul/25 16:21:35] INFO [telemetry_logging.wrapper:91] SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
[15/Jul/25 16:21:35] INFO [session.submit:1053] Creating training-job with name: 

2025-07-15 23:21:36 Starting - Starting the training job...
2025-07-15 23:21:52 Starting - Preparing the instances for training...
2025-07-15 23:22:13 Downloading - Downloading input data...
2025-07-15 23:23:04 Downloading - Downloading the training image......
2025-07-15 23:23:54 Training - Training image download completed. Training in progress../miniconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[2025-07-15 23:23:59.943 ip-10-0-123-70.us-west-2.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-07-15 23:23:59.965 ip-10-0-123-70.us-west-2.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2025-07-15:23:24:00:INFO] Imported framework sagemaker_xgboost_container.training
[2025-07-15:23:24:00:INFO] Failed to parse hyperpara

[15/Jul/25 16:27:02] INFO [wallet_modeler.train_model:166] Training completed. Model stored at: s3://wallet-training-data/sagemaker-models/dda-897-td-col-ordering/wallet-xgb-dda-897-td-col-ordering-231107-20250715-162135/output/model.tar.gz


{'model_uri': 's3://wallet-training-data/sagemaker-models/dda-897-td-col-ordering/wallet-xgb-dda-897-td-col-ordering-231107-20250715-162135/output/model.tar.gz',
 'training_job_name': 'wallet-xgb-dda-897-td-col-ordering-231107-20250715-162135',
 'date_suffix': '231107'}

#### get existing model uri

In [23]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


modeler = wm.WalletModeler(sage_wallets_config, s3_uris)
date_suffix = date_suffixes[0]


model = modeler.load_existing_model(date_suffix)

[15/Jul/25 17:05:10] INFO [wallet_modeler.load_existing_model:254] Loaded most recent model (timestamp: 20250715-162135): s3://wallet-training-data/sagemaker-models/dda-897-td-col-ordering/wallet-xgb-dda-897-td-col-ordering-231107-20250715-162135/output/model.tar.gz


In [24]:
predictions = modeler.predict_with_model()

[15/Jul/25 17:05:12] INFO [image_uris._processor:530] Ignoring unnecessary instance type: None.
[15/Jul/25 17:05:12] INFO [session.submit:4105] Creating model with name: wallet-model-dda-897-td-col-ordering
[15/Jul/25 17:05:13] INFO [wallet_modeler.predict_with_model:325] Starting batch transform job: wallet-scoring-231107-170513
[15/Jul/25 17:05:13] INFO [wallet_modeler.predict_with_model:326] Using model: wallet-model-dda-897-td-col-ordering
[15/Jul/25 17:05:13] INFO [wallet_modeler.predict_with_model:327] Input data: s3://wallet-training-data/training-data-preprocessed/dda-897-td-col-ordering/231107/val_cw_crypto_net_gain_crypto_inflows_winsorized.csv
[15/Jul/25 17:05:13] INFO [wallet_modeler.predict_with_model:328] Output path: s3://wallet-training-data/validation-data-scored/
[15/Jul/25 17:05:13] INFO [session.submit:3962] Creating transform job with name: wallet-scoring-231107-170513


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 predictions = modeler.predict_with_model()                                                   │
│   2                                                                                              │
│                                                                                                  │
│ /Users/jeremymeadow/DreamsData/GitHub/sagemaker/notebooks/..//src/sagemaker_wallets/wallet_model │
│ er.py:331 in predict_with_model                                                                  │
│                                                                                                  │
│   328 │   │   logger.info(f"Output path: {output_path}")                                         │
│   329 │   │                                                                                      │
│   330 │   │   # Start batch transform                                                            │
│ ❱ 331 │   │   transformer.transform(                                                             │
│   332 │   │   │   data=date_uris['val'],                                                         │
│   333 │   │   │   content_type='text/csv',                                                       │
│   334 │   │   │   split_type='Line',                                                             │
│                                                                                                  │
│ /Users/jeremymeadow/DreamsData/.venv/lib/python3.11/site-packages/sagemaker/workflow/pipeline_co │
│ ntext.py:346 in wrapper                                                                          │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                                                            │
│                                                                                                  │
│ /Users/jeremymeadow/DreamsData/.venv/lib/python3.11/site-packages/sagemaker/transformer.py:302   │
│ in transform                                                                                     │
│                                                                                                  │
│   299 │   │   │   sagemaker_session=self.sagemaker_session,                                      │
│   300 │   │   )                                                                                  │
│   301 │   │                                                                                      │
│ ❱ 302 │   │   self.latest_transform_job = _TransformJob.start_new(                               │
│   303 │   │   │   self,                                                                          │
│   304 │   │   │   data,                                                                          │
│   305 │   │   │   data_type,                                                                     │
│                                                                                                  │
│ /Users/jeremymeadow/DreamsData/.venv/lib/python3.11/site-pa

### DDA 884 devspace